In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [3]:
data = pd.read_csv("/content/Medicine_Details.csv")
label_encoder = LabelEncoder()
data['Medicine_ID'] = label_encoder.fit_transform(data['Medicine Name'])
data['Condition_ID'] = label_encoder.fit_transform(data['Uses'].fillna("Unknown"))
scaler = MinMaxScaler()
data[['Excellent Review %', 'Average Review %', 'Poor Review %']] = scaler.fit_transform(
    data[['Excellent Review %', 'Average Review %', 'Poor Review %']]
)


In [4]:
features = data[['Medicine_ID', 'Condition_ID', 'Excellent Review %', 'Average Review %', 'Poor Review %']]
target = data['Medicine_ID']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [5]:
model = Sequential([
    Dense(256, activation='relu', input_dim=features.shape[1]),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(data['Medicine_ID'].unique()), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)


Epoch 1/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 12s 42ms/step - accuracy: 0.0000e+00 - loss: 27.6831 - val_accuracy: 0.0000e+00 - val_loss: 9.4362
Epoch 2/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 1.7056e-04 - loss: 9.3891 - val_accuracy: 0.0000e+00 - val_loss: 9.5850
Epoch 3/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 6.4265e-04 - loss: 9.2720 - val_accuracy: 0.0000e+00 - val_loss: 9.7283
Epoch 4/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 8.2976e-05 - loss: 9.2143 - val_accuracy: 0.0000e+00 - val_loss: 9.8675
Epoch 5/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 2.7336e-05 - loss: 9.1670 - val_accuracy: 0.0000e+00 - val_loss: 10.0027
Epoch 6/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 5.9668e-04 - loss: 9.1283 - val_accuracy: 0.0000e+00 - val_loss: 10.1339
Epoch 7/200
237/237 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 2.8687e-04 - loss: 9.0842 - val_accuracy: 0.0000e+00 - val_loss: 10.2615
Epoch 8/200
237/237 ━━━━━━━━

In [8]:
def recommend_medicine(condition_id, top_n=5):
    condition_vector = np.zeros((1, features.shape[1]))
    condition_vector[0, 1] = condition_id
    predictions = model.predict(condition_vector)
    top_medicines = predictions.argsort()[0][-top_n:][::-1]
    return data[data['Medicine_ID'].isin(top_medicines)][['Medicine Name', 'Uses']]


In [9]:
def get_condition_id(user_input):
    matched_conditions = data[data['Uses'].str.contains(user_input, case=False, na=False)]
    if matched_conditions.empty:
        return f"No matches found for condition: {user_input}"
    return matched_conditions['Condition_ID'].iloc[0]


In [10]:
def interactive_recommendation():
    print("Welcome to the Medicine Recommender System!")
    user_condition = input("Enter the condition you need recommendations for (e.g., headache, fever): ")
    condition_id = get_condition_id(user_condition)
    if isinstance(condition_id, str):
        print(condition_id)
    else:
        recommendations = recommend_medicine(condition_id)
        print("\nTop Recommended Medicines for Your Condition:")
        if recommendations.empty:
            print("No medicines found for the given condition.")
        else:
            print(recommendations)


In [14]:
interactive_recommendation()


Welcome to the Medicine Recommender System!
Enter the condition you need recommendations for (e.g., headache, fever): skin infection
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Top Recommended Medicines for Your Condition:
              Medicine Name                                               Uses
4253          Lulifin Cream                Treatment of Fungal skin infections
4429  Livoluk Oral Solution  Treatment of ConstipationTreatment of Hepatic ...
4458          Lulifin Cream                Treatment of Fungal skin infections
4490          Lulifin Cream                Treatment of Fungal skin infections
4531          Lulifin Cream                Treatment of Fungal skin infections
4613  Livoluk Oral Solution  Treatment of ConstipationTreatment of Hepatic ...
4656          Lumycan Cream                Treatment of Fungal skin infections
4827  Livoluk Oral Solution  Treatment of ConstipationTreatment of Hepatic ...
4992          Lumycan Cream                Treatment of Fungal skin inf